Genetski algoritam je primer p metaheuristike, kod kojih imamo populaciju rešenja za razliku od s metaheuristika, gde smo imali samo jedno rešenje.
Genetski algoritam je osmišljen po ugledu na biološku evoluciju.
Sada evoluiramo rešenja nekog optimizacionog problema.
Svaka jedinka predstavlja jedno rešenje i ima svoj kod, paralela bi bila npr. DNK, kao i funkciju prilagođenosti (fitness).
I ovde je ideja da najprilagođenije jedinke opstaju.
Formiramo populaciju jedinki koje evoluiramo kroz određeni broj generacija, odnosno iteracija.
Postoje i druge opcije, ali ovde smo se odlučili da u novoj generaciji formiramo novu populaciju i da ona zameni postojeću.
Tri glavne komponente genetskog algoritma su:
1. selekcija - odabir jedinki za ukrštanje na osnovu njihove prilagođenosti. Dve osnovne varijante selekcije su:
    - turnirska - odabir k slučajnih jedinki iz populacije i "turnirsko takmičenje" gde pobeđuje najprilagođenija od izabranih jedniki.
    Razmisliti šta se dešava kada je k = 1, a šta kada je k = veličini populacije.
    - ruletska - raspoređivanje svih jedinki na pristrasan ruletski točak gde je veličina isečka koji jedinka zauzima proporcionalna njenom fitnesu. Dakle, bolje prilagođene jedinke imaju veću verovatnoću da budu izabrane.
    Razmisliti o situaciji gde je npr. jedna jedinka ubedljivo bolja od ostalih. Kako biste rešili ovakav problem?
2. ukrštanje - formiranje novih jedinki na osnovu postojećih - u ovom slučaju formiramo dva potomka od dva roditelja. Nekoliko osnovnih varijanti ukrštanja su:
    - jednopoziciono ukrštanje - odabir jedne slučajne tačke prekida u kodu roditeljskih jedinki i unakrsno ukrštanje kodova
    - višepoziciono ukrštanje - analogno
    - uniformno ukrštanje - za svaki "gen", u ovom slučaju bit u rešenju, bacamo novčić i na osnovu toga biramo iz kog roditelja ćemo kopirati trenutni "gen" u dete
3. mutacija - sa određenom (malom) verovatnoćom na slučajan način unosimo promenu u gene deteta - u ovom slučaju negiramo bit

In [ ]:
import random


class Individual:
    """
    Individual of genetic algorithm
    code - represents one solution
    fitness - quality of the solution
    """
    def __init__(self, facility_cost, user_facility_cost):
        self.code = [random.random() < 0.5 for _ in range(len(facility_cost))]
        self.facility_cost = facility_cost
        self.user_facility_cost = user_facility_cost

    def calc_fitness(self):
        """UFLP cost - negative because we want to minimize cost, but maximize fitness"""
        if not any(self.code):
            return -float('inf')
        built_schools = sum(x for i, x in enumerate(self.facility_cost) if self.code[i])
        user_to_nearest_school = sum(min(cost for i, cost in enumerate(user) if self.code[i]) for user in self.user_facility_cost)
        self.fitness = -(built_schools + user_to_nearest_school)

In [ ]:
def selection(population: list[Individual], k: int):
    """
    Tournament selection
    1. Choose random k individuals from the population
    2. The best of those k wins the tournament and is selected for crossover

    TODO: roulette (rang) selection
    """
    participants = random.sample(population, k)
    return max(participants, key=lambda x: x.fitness)

In [ ]:
def crossover(parent1, parent2, child1, child2):
    """
    Single point crossover - changes children in place
    
    TODO:
    1. n-point crossover - self-explanatory
    2. uniform - for each bit flip a coin
    """
    breakpoint = random.randrange(1, len(parent1.code))
    child1.code[:breakpoint] = parent1.code[:breakpoint]
    child1.code[breakpoint:] = parent2.code[breakpoint:]

    child2.code[:breakpoint] = parent2.code[:breakpoint]
    child2.code[breakpoint:] = parent1.code[breakpoint:]

In [ ]:
def mutation(child: Individual, p: float):
    """
    With probability p flip each bit
    """
    # child.code = [not x if random.random() < p else x for x in child.code]
    for i in range(len(child.code)):
        if random.random() < p:
            child.code[i] = not child.code[i]

In [ ]:
from copy import deepcopy


def ga(facility_cost, user_facility_cost, population_size, num_generations, tournament_size, mutation_prob, elitism_size):
    """
    Genetic Algorithm for UFLP
    
    population_size - number of individuals in a generation
    num_generations - number of iterations of algorithm
    elitism_size - number of best individuals that will be copied to next generation
                 - to avoid losing them due to unlucky crossovers and mutations
    """
    population = [Individual(facility_cost, user_facility_cost) for _ in range(population_size)]
    new_population = [Individual(facility_cost, user_facility_cost) for _ in range(population_size)]

    # parity of elitism_size and population_size must be the same
    # since we are going two by two in the inner loop
    if elitism_size % 2 != population_size % 2:
        elitism_size += 1

    for it in range(num_generations):
        population.sort(key=lambda x: x.fitness)
        new_population[:elitism_size] = population[:elitism_size]
        for i in range(elitism_size, population_size, step=2):
            # we should fill new_pop[i] and new_pop[i+1]
            parent1 = selection(population, tournament_size)
            # TODO think about repetitions
            parent2 = selection(population, tournament_size)
            
            crossover(parent1, parent2, new_population[i], new_population[i+1])

            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)

            new_population[i].calc_fitness()
            new_population[i+1].calc_fitness()
        
        population = deepcopy(new_population)

    return max(population, key=lambda x: x.fitness)

In [ ]:
# TODO test